# Exemplary analysis of the PBMC3K single-cell RNA dataset

Important version: `umap-learn==0.5.1`

In [ ]:
from typing import Optional, List
import csv
import os
import math
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import mygene

from anndata import AnnData
import scanpy as sc
from numba import NumbaWarning

# keggtool imports
from keggtools import (
    Pathway,
    Enrichment,
    EnrichmentResult,
    Resolver,
    Storage,
    Renderer,
    IMMUNE_SYSTEM_PATHWAYS,
)

# Used folders
rawdata_dir = os.path.join(os.getcwd(), "rawdata")
figure_dir = os.path.join(os.getcwd(), "figures")

# Global settings
sc.settings.verbosity = 0
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

# Ignore all warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=NumbaWarning)

In [ ]:
adata: AnnData = sc.read_10x_mtx(rawdata_dir, var_names="gene_symbols", cache=False)
adata.var_names_make_unique()

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
adata = adata[adata.obs["n_genes_by_counts"] < 2500, :]
adata = adata[adata.obs["pct_counts_mt"] < 5, :]

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
adata.raw = adata
adata = adata[:, adata.var["highly_variable"]]

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata)

In [ ]:
# TODO: plot marker genes of cluster
sc.pl.umap(adata, color=['CST3', 'NKG7'])

In [ ]:
sc.pl.violin(adata, ['CST3', 'NKG7', 'PPBP'], groupby='leiden', rotation=90.0)

### Rename leiden clusters

In [ ]:
from typing import List


new_cluster_names: List[str] = [
    'CD4+ T-cells',
    'CD14+ Monocytes',
    'B-cells',
    'CD8+ T-cells',
    'NK cells',
    'FCGR3A+ Monocytes',
    'Dendritic cells',
    'Megakaryocytes'
]
adata.rename_categories('leiden', new_cluster_names)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,8))
ax.set_aspect('equal', adjustable='box')

sc.pl.umap(adata, color='leiden', legend_loc='right margin', title='', frameon=True, ax=ax, show=False)

fig.tight_layout(pad=3.0)
fig.savefig(os.path.join(figure_dir, "figure2.png"))

### Differential analysis results to pandas dataframe

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', n_genes=100)

In [ ]:
# Print out marker genes per cluster
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

In [ ]:
def export_uns_data(
    adata_obj: AnnData,
    descriptor: Optional[str] = "rank_genes_groups",
    extract_cols=["names", "pvals", "pvals_adj", "logfoldchanges"],
    ) -> pd.DataFrame:
    """
    Helper function to export marker gene data from AnnData uns to pandas dataframe.

    """


    n_clusters = len(adata_obj.uns[descriptor]["names"][0])
    n_clusters

    data = {"cluster": []}
    for col in extract_cols:
        data[col] = []

    results_df = pd.DataFrame(data)

    items_per_cluster = len(np.array([list(i) for i in np.array(adata_obj.uns[descriptor][extract_cols[0]])])[:,0])

    for cluster in range(0, n_clusters):

        data = {"cluster": []}
        for col in extract_cols:
            data[col] = []

        df = pd.DataFrame(data)

        df["cluster"] = pd.array([cluster] * items_per_cluster, dtype="Int32")

        for col in extract_cols:
            cluster_vars = np.array([list(i) for i in np.array(adata_obj.uns[descriptor][col])])[:,cluster]
            df[col] = cluster_vars

        results_df = results_df.append(df)

    return results_df

In [ ]:
diffexp_df = export_uns_data(adata)
diffexp_df.head()

### Perform KEGGTOOLS enrichment analysis

In [ ]:
organism_id: str = "hsa"

# Analysed Cluster 4 (NK cells)
cluster: int = 4

In [ ]:
diffexp_df = diffexp_df[diffexp_df["cluster"] == cluster]
diffexp_df.head()

### Convert to list of entez ids using mygene

In [ ]:
# Resolve list of gene symbols to list of entrez gene ids

from warnings import warn
import mygene

mygene_info = mygene.MyGeneInfo()

query_result = mygene_info.querymany(list(diffexp_df["names"]), scopes="symbol", species="human")

entrz_gene_list: List[str] = []
not_found_gene_list: List[str] = []

for item in query_result:
    if "entrezgene" in item:
        # Append entrez id to list
        entrz_gene_list.append(str(item["entrezgene"]))

        # Add entrez id to dataframe
        diffexp_df.loc[diffexp_df["names"] == item["query"], "entrez"] = str(item["entrezgene"])
    else:

        # Append not found genes to list
        not_found_gene_list.append(item["query"])
        diffexp_df.loc[diffexp_df["names"] == item["query"], "entrez"] = 0

if len(not_found_gene_list) > 0:
    warn(f"In total {len(not_found_gene_list)} were not found in mygene query.", category=UserWarning)


In [ ]:
from typing import List


resolver: Resolver = Resolver()
pathway_list: List[Pathway] = []

# Download all immune system pathways
for number, _ in IMMUNE_SYSTEM_PATHWAYS.items():
    pathway_list.append(resolver.get_pathway(organism=organism_id, code=number))


In [ ]:
analysis: Enrichment = Enrichment(pathways=pathway_list)
analysis_result: List[EnrichmentResult] = analysis.run_analysis(gene_list=list(diffexp_df["entrez"]))
result_df: pd.DataFrame = analysis.to_dataframe()

# Filter out pathways with no genes found
result_df = result_df[result_df["study_count"] > 0]

# Print out result
result_df.head()

### Plot results of enrichment analysis

In [ ]:
import matplotlib.pyplot as plt
import math

plt.figure(figsize=(8, 5), dpi=300)
scatter = plt.scatter(x=result_df["study_count"] / result_df["pathway_genes"] * 100,
                      y=result_df["pathway_title"],
                      c=[-math.log10(x) for x in result_df["pvalue"]],
                      cmap="coolwarm")

cbar = plt.colorbar()
cbar.set_label("- log10(p value)")

plt.grid(b=None)
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, "figure4.png"), bbox_inches='tight')
plt.show()

### Plot pathway

* "Natural killer cell mediated cytotoxicity" (hsa:04650) show a significant p value

In [ ]:
pathway: Pathway = resolver.get_pathway(organism=organism_id, code="04612")

In [ ]:
# diffexp_df[["entrez"]] = diffexp_df[["entrez"]].astype(int)
overlay: dict = dict(zip(list(diffexp_df["entrez"]), list(diffexp_df["logfoldchanges"])))

In [ ]:
renderer: Renderer = Renderer(kegg_pathway=pathway, gene_dict=overlay, cache_or_resolver=resolver.storage)
renderer.render(resolve_unlabeled_genes=True, truncate_gene_list=5)

In [ ]:
# Save dot string to file
with open(os.path.join(figure_dir, "figure5.dot"), "w") as file_obj:
    file_obj.write(renderer.to_string())

# Save binary data to file
renderer.to_file(filename=os.path.join(figure_dir, "figure5.png"), extension="png")

In [ ]:
# Display image
from IPython.display import Image, display
img: Image = Image(os.path.join(figure_dir, "figure5.png"))
display(img)